In [114]:
import json
import urllib.request
from PIL import Image
import os
import math
import pandas as pd
import numpy as np

In [94]:
class GoogleMapsLayers:
    ROADMAP = "v"
    TERRAIN = "p"
    ALTERED_ROADMAP = "r"
    SATELLITE = "s"
    TERRAIN_ONLY = "t"
    HYBRID = "y"


class GoogleMapDownloader:
    """
        A class which generates high resolution google maps images given
        a longitude, latitude and zoom level
    """

    def __init__(self, lat, lng, zoom=12, layer=GoogleMapsLayers.ROADMAP, code=''):
        """
            GoogleMapDownloader Constructor
            Args:
                lat:    The latitude of the location required
                lng:    The longitude of the location required
                zoom:   The zoom level of the location required, ranges from 0 - 23
                        defaults to 12
        """
        self._lat = lat
        self._lng = lng
        self._zoom = zoom
        self._layer = layer
        self.tiles = 2
        if code != '':
            self.code = f'{code}_'
        else:
            self.code = ''

    def getXY(self):
        """
            Generates an X,Y tile coordinate based on the latitude, longitude
            and zoom level
            Returns:    An X,Y tile coordinate
        """

        tile_size = 256

        # Use a left shift to get the power of 2
        # i.e. a zoom level of 2 will have 2^2 = 4 tiles
        numTiles = 1 << self._zoom

        # Find the x_point given the longitude
        point_x = (tile_size / 2 + self._lng * tile_size / 360.0) * numTiles // tile_size

        # Convert the latitude to radians and take the sine
        sin_y = math.sin(self._lat * (math.pi / 180.0))

        # Calulate the y coorindate
        point_y = ((tile_size / 2) + 0.5 * math.log((1 + sin_y) / (1 - sin_y)) * -(
        tile_size / (2 * math.pi))) * numTiles // tile_size

        return int(point_x), int(point_y)

    def generateImage(self, **kwargs):
        """
            Generates an image by stitching a number of google map tiles together.
            Args:
                start_x:        The top-left x-tile coordinate
                start_y:        The top-left y-tile coordinate
                tile_width:     The number of tiles wide the image should be -
                                defaults to 5
                tile_height:    The number of tiles high the image should be -
                                defaults to 5
            Returns:
                A high-resolution Goole Map image.
        """

        start_x = kwargs.get('start_x', None)
        start_y = kwargs.get('start_y', None)
        tile_width = kwargs.get('tile_width', 5)
        tile_height = kwargs.get('tile_height', 5)

        # Check that we have x and y tile coordinates
        if start_x == None or start_y == None:
            start_x, start_y = self.getXY()

        # Determine the size of the image
        width, height = 256 * tile_width, 256 * tile_height
        
        
        
        start_xy_lst = []
        for ix in range(self.tiles):
            for iy in range(self.tiles):
                s_x = start_x + ix * tile_width
                s_y = start_y + iy * tile_height
                start_xy_lst.append((s_x, s_y))     
    
        for i, (start_x, start_y) in enumerate(start_xy_lst):
            
            # Create a new image of the size require
            map_img = Image.new('RGB', (width, height))

            for x in range(0, tile_width):
                for y in range(0, tile_height):
                    url = f'https://mt0.google.com/vt?lyrs={self._layer}&x=' + str(start_x + x) + '&y=' + str(start_y + y) + '&z=' + str(
                        self._zoom)
                    current_tile = str(x) + '-' + str(y)
                    urllib.request.urlretrieve(url, current_tile)

                    im = Image.open(current_tile)
                    map_img.paste(im, (x * 256, y * 256))

                    os.remove(current_tile)
            filename = f'{self.code}{start_x}_{tile_width}_{start_y}_{tile_height}_{self._zoom}.jpg'
            filepath = os.path.join(os.getcwd(), filename)
            map_img.save(filepath)
            print(f'{filename} saved')
        return map_img
    
    def get_xy_list(self):
        start_x, start_y = self.getXY()
        width = 3
        lst = []
        for ix in range(2):
            for iy in range(2):
                s_x = start_x + ix * width
                s_y = start_y + iy * width
                lst.append((s_x, s_y))

        lst


In [127]:
lat_long = [
    # hampstead heath
    (51.5638, -0.1647, 'hh'),
    # teddington
    (51.4497, -0.3061, 'td'),
    # richmond
    (51.4589, -0.2977, 'rc'),
    # kingston
    (51.4202, -0.2861, 'kn'),
    # victoria park
    (51.5384, -0.0357, 'vp'),
    # hackney
    (51.5537, -0.0448, 'hn'),
    # walthamstow
    (51.5848, -0.0118, 'wm'),
    # bond street
    (51.5127, -0.152, 'bs'),
    # swiss cottage
    (51.5433, -0.1647, 'sc'),
    # streatham
    (51.4311, -0.1231, 'st')
    # eltham
    (51.4403, 0.0273, 'el'),
    # stepney green
    (51.5180, -0.0474, 'sg'),
    # islington
    (51.5366, -0.1014, 'is'),
    # caledonian road
    (51.5410, -0.1201, 'cr'),
    # finchley
    (51.5833, -0.1727, 'fn'),
    # camden
    (51.5378, -0.1391, 'cd')
    # clapham
    (51.4578, -0.1630, 'cl')
    # richmond park
    (51.4474, -0.2777, 'rp'),
    # weybridge
    (51.3601, -0.4458, 'wb'),
    # kingston gate
    (51.4290, -0.2820, 'kg')
    # norwood grove
    (51.4208, -0.1166, 'ng')
    # beckenham park
    (51.4209, -0.0172, 'bp'),
    # Sundridge
    (51.4198, 0.0274, 'sr')
]

zoom = 19

for lat, long, code in lat_long:
    gmd = GoogleMapDownloader(lat, long, zoom, GoogleMapsLayers.SATELLITE, code)
    print("The tile coorindates are {}".format(gmd.getXY()))
    img = gmd.generateImage(tile_width=4, tile_height=4)

The tile coorindates are (262118, 174541)
bp_262118_4_174541_4_19.jpg saved
bp_262118_4_174545_4_19.jpg saved
bp_262122_4_174541_4_19.jpg saved
bp_262122_4_174545_4_19.jpg saved
The tile coorindates are (262183, 174543)
sr_262183_4_174543_4_19.jpg saved
sr_262183_4_174547_4_19.jpg saved
sr_262187_4_174543_4_19.jpg saved
sr_262187_4_174547_4_19.jpg saved
